In [2]:
import bz2
import os
from tqdm.notebook import tqdm
import numpy as np
import pyodbc

In [3]:
import helpers
#import importlib
#importlib.reload(helpers)

### Загрузка id сессий для каждого файла

In [11]:
file_list = []
for file in tqdm(os.listdir("2017"), total=1905):
    if file.endswith(".bz2"):
        file_list.append(file)

In [279]:
log_id = log_ids['part-m-00135.bz2'][1]
file_list = file_list[len(log_ids):]
for file in tqdm(file_list, total=len(file_list)):
    if file.endswith(".bz2"):
        filename = os.path.join("2017", file)
        with bz2.open(filename, "rb") as f:
            lines = f.readlines()
        log_ids[file] = (log_id, log_id + len(lines))
        log_id = log_id + len(lines)

In [285]:
helpers.save_obj(log_ids, 'log_ids')

### Загрузка queries из click data

In [ ]:
def process_query_file(cnxn, filename, log_id):
    prev_query = ''
    cnt_query = 0
    cursor = cnxn.cursor()
    with bz2.open(os.path.join("2017", filename), "rb") as f:
        lines = f.readlines()
    for line_ind, line in enumerate(tqdm(lines, total=len(lines))):
        line = line.decode('utf-8').rstrip('\n')
        query = line.split("@")[0]
        if prev_query != query:
            cnt_query = cnt_query + 1
            cursor.execute("INSERT INTO dbo.log_query (query, filename, log_id_start) values(?,?,?)", 
                           query, filename, log_id)
            prev_query = query
        log_id = log_id + 1
        if cnt_query % 10_000:
            cnxn.commit()
    cnxn.commit()
    cursor.close()
    return log_id

In [976]:
def process_query_file_thread(filename):
    with helpers.get_sql_conn() as cnxn:
        bz2
        process_query_file(cnxn, filename, log_id)
    # счетчик должен атомарно обновиться
    with lock:
        pbar.update(1)

In [179]:
log_ids = helpers.load_obj('log_ids')

In [5]:
with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
    cursor.execute("SELECT [Filename] FROM [ir].[dbo].[log_query] group by [Filename]")
    rows = cursor.fetchall()

In [14]:
processed_files = {row[0]: 0 for row in rows}

In [33]:
rest_files = []
for file in file_list:
    if file not in processed_files:
        rest_files.append(file)

In [37]:
from multiprocessing.dummy import Pool, Lock, Queue
with Pool(processes=8) as pool, tqdm(total=len(rest_files)) as pbar:
    lock = pbar.get_lock()
    pool.map(process_query_file_thread, rest_files)

### Загрузка сессий, где есть url'и из url.data 

USE ir;  
GO  
IF EXISTS (SELECT name FROM sys.indexes  
            WHERE name = N'ix_log_query_log_id_start')   
    DROP INDEX ix_log_query_log_id_start ON dbo.log_query;   
GO  
CREATE NONCLUSTERED INDEX ix_log_query_log_id_start   
    ON dbo.log_query (log_id_start);   
GO  

In [4]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [5]:
def get_url_wo_scheme(url):
    prefix_scheme_list = ['http://', 'https://', 'ftp://']
    for prefix_scheme in prefix_scheme_list:
        if url.find(prefix_scheme) == 0:
            url = url[len(prefix_scheme):]
            break
    return url

In [6]:
def get_url_wo_www(url):
    prefix_www = 'www.'
    if url.find(prefix_www) == 0:
        url = url[len(prefix_www):]
    return url

In [7]:
def get_clear_url(url):
    url = get_url_wo_scheme(url)
    url = get_url_wo_www(url)
    return url

In [8]:
from urllib.parse import urlparse
def get_host(url, wo_www=True):
    if wo_www:
        url = get_url_wo_www(url)
    return urlparse(f"http://{url}").hostname.lower()

In [8]:
url_data_df = pd.read_csv('url.data', header=None, sep='\t')
url_data_df.rename(columns={0: "doc_id", 1: "doc_url"}, inplace=True)

In [12]:
tqdm.pandas()

In [13]:
url_data_df['doc_url_corrected'] = url_data_df.progress_apply(lambda row: get_clear_url(row.doc_url), axis=1)

In [14]:
url_data_df['host'] = url_data_df.progress_apply(lambda row: urlparse(f"http://{row.doc_url_corrected}").hostname, axis=1)

In [15]:
hosts_unique = np.unique(url_data_df['host'], return_counts=False)

In [70]:
with open('host.data', 'w') as fout:
    fout.write('host_id,host\n')
    for host_id, host in enumerate(hosts_unique):
        fout.write('{0},{1}\n'.format(host_id + 1, host))

USE ir;  
GO  
ALTER TABLE dbo.host_data  
ADD CONSTRAINT PK_host_data_host_id PRIMARY KEY CLUSTERED (host_id);  

USE ir;  
GO  
IF EXISTS (SELECT name FROM sys.indexes  
		   WHERE name = N'ix_host_data_host')  
		   DROP INDEX ix_host_data_host ON dbo.host_data;  
GO  
CREATE NONCLUSTERED INDEX ix_host_data_host ON dbo.host_data(host);  
GO

In [16]:
host_map = {host: host_id + 1 for host_id, host in enumerate(hosts_unique)}

In [17]:
host_set = set(host_map.keys())

In [18]:
url_data_df['host_id'] = url_data_df.progress_apply(lambda row: host_map[row.host], axis=1)

In [93]:
url_data_df[['doc_id', 'doc_url', 'doc_url_corrected', 'host_id']].to_csv('url_data_corrected.csv', index=False, sep='\t')

ALTER TABLE dbo.url_data  
ADD CONSTRAINT FK_url_host FOREIGN KEY ([host_id])  
REFERENCES dbo.host_data ([host_id]);  

In [19]:
url_map = {row.doc_url: row.doc_id for _, row in url_data_df.iterrows()}

In [20]:
url_set = set(url_map.keys())

**Замечание по поводу данных:**  
Документы 11177 (www.bluestacks.com/RU/index.html) и  
11179 (www.bluestacks.com/ru/index.html) идентичны по тексту, отличаются лишь регистр в пути url.    
При этом в файле с ассесорскими оценками: 6051	11177	4, 6051	11179	3

In [22]:
url_data_df.to_csv('url_data_df.csv', index=False, sep='\t')

#### Загрузка

In [9]:
#подмесы
url_ext_list = ['health_consultations', 'images', 'video', 'weather', 'answer', 'news', 
                'spritze.horoscope-common', 'map', 'recipes', 'osmino', 'app',
                'music', 'youla_web', 'person', 'afisha', 'newstext', 'howtos',
                'drugs', 'NONE', 'spritze.symptoms', 'torgs', 'facts']
url_ext_list = [(s, len(f',{s}')) for s in url_ext_list]

In [10]:
click_data_df = pd.read_csv('click_data_df.csv', sep='\t')
click_data_df.sort_values(by=['query_id', 'filename', 'log_id_start'], inplace=True)

In [11]:
train_df = pd.read_csv('train_sphinx_df.csv', sep='\t')[['doc_id', 'query_id']]
test_df = pd.read_csv('test_sphinx_df.csv', sep='\t')[['doc_id', 'query_id']]

In [12]:
import bz2
import os
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

In [19]:
import helpers
#import importlib
#importlib.reload(helpers)

In [13]:
log_ids = helpers.load_obj('log_ids')

In [14]:
url_data_df = pd.read_csv('url_data_df.csv', sep='\t')

In [15]:
hosts_unique = np.unique(url_data_df['host'], return_counts=False)
host_map = {host: host_id + 1 for host_id, host in enumerate(hosts_unique)}
host_set = set(host_map.keys())
url_map = {row.doc_url: row.doc_id for _, row in url_data_df.iterrows()}
url_set = set(url_map.keys())

In [16]:
def get_ext_url(url, url_ext_list):
    pos_list = np.array(list(map(lambda url_ext: url.rfind(f',{url_ext[0]}'), url_ext_list)))
    indices = np.argwhere(pos_list >= 0)
    url_len = len(url)
    if len(indices) > 0:
        for i in indices:
            i = i[0]
            pos = pos_list[i]
            suffix_len = url_ext_list[i][1]
            url_ext = url_ext_list[i][0]
            if pos >= 0 and pos + suffix_len == url_len:
                return url[:pos], url_ext
            else:
                continue
    return url, None

def get_url_ext_list(url_str, url_ext_list=url_ext_list):
    url_str = url_str.replace(',https://', ',http://')
    url_str = url_str.replace(',ftp://', ',http://')
    url_list = url_str.split(',http://')
    ext_list = []
    for url in url_list:
        found_url_ext_list = []
        cleared_url = url
        for _ in url_ext_list:
            cleared_url, url_ext = get_ext_url(cleared_url, url_ext_list)
            if url_ext:
                found_url_ext_list.append(url_ext)
            else:
                break
        ext_list.append(cleared_url)
        ext_list.extend(found_url_ext_list[::-1])
    ext_list[0] = get_url_wo_scheme(ext_list[0])
    return ext_list

In [17]:
def parse_log_line(line, url_set=None, host_set=None):
    query = line.split("@")[0]
    line = line[len(query) + 1:]
    data = line.split('\t')
    geoId = data[0]
    show_list = get_url_ext_list(data[1])
    show_host_list = list(map(lambda x: get_host(x), show_list))
    if url_set:
        if not set(show_list) & url_set:
            if host_set:
                if not set(show_host_list) & host_set:
                    return False, None
            else:
                return False, None
        
    click_list = get_url_ext_list(data[2])
    if len(click_list) == 0:
        return False, None
    time_list = list(map(int, data[3].split(',')))
    ordered_click_dict = {click_list[i]: time_list[i] for i in np.array(time_list).argsort()}
    ordered_click_list = list(ordered_click_dict.keys())
    ordered_click_time_list = list(ordered_click_dict.values())
    
    show_dict = {}
    for i, url in enumerate(show_list):
        is_click = 0
        url_id = None
        host_id = None
        if url in url_map:
            url_id = url_map[url]
        host = show_host_list[i]
        if host in host_map:
            host_id = host_map[host]
        if url in ordered_click_dict:
            is_click = 1
        show_dict[url] = (i + 1, url_id, host_id, is_click)
        
    fclick_pos = show_dict[ordered_click_list[0]][0]
    lclick_pos = show_dict[ordered_click_list[-1]][0]
    
    click_dict = {}
    for i, (url, t) in enumerate(ordered_click_dict.items()):
        url_id = None
        host_id = None
        if url in url_map:
            url_id = url_map[url]
        host = get_host(url)
        if host in host_map:
            host_id = host_map[host]
        t_start = int(t / 1000)
        if i < len(ordered_click_dict) - 1:
            t_end = int(ordered_click_time_list[i + 1] / 1000)
        else:
            t_end = None
        click_dict[url] = (i + 1, url_id, host_id, t_start, t_end)
    
    t_start_session = int(ordered_click_time_list[0] / 1000)
    t_end_session = int(ordered_click_time_list[-1] / 1000)
    return True, query, geoId, show_dict, click_dict, \
           fclick_pos, lclick_pos, \
           t_start_session, t_end_session

In [18]:
def process_log_line(cursor, log_id, query_id, line,
                     url_set=None,
                     host_set=None,
                     session_tbl='dbo.log_session_tmp', 
                     show_tbl='dbo.log_show_tmp',
                     click_tbl='dbo.log_click_tmp', 
                     check_session_tbl=True):
    if check_session_tbl:
        cursor.execute(f"select top 1 [id] FROM [ir].[dbo].[log_session] where [id] = {log_id}")
        row = cursor.fetchone()
        if row:
            return False
        
    res, *data = parse_log_line(line, url_set=url_set, host_set=host_set)
    if not res:
        return False

    query = data[0]
    geo_id = data[1]
    show_dict = data[2]
    click_dict = data[3]
    fclick_pos = data[4]
    lclick_pos = data[5]
    t_start_session = data[6]
    t_end_session = data[7]

    cursor.execute(f"insert into {session_tbl} \
                     (id, geo_id, filename, query_id, t_start, t_end, fclick_pos, lclick_pos) \
                     values(?,?,?,?,?,?,?,?)", 
                     log_id, geo_id, filename, 
                     query_id, t_start_session, t_end_session,
                     fclick_pos, lclick_pos)
    for url, (pos, url_id, host_id, is_click) in show_dict.items():
        cursor.execute(f"insert into {show_tbl} (session_id, pos, url_id, is_click,\
                         host_id) values(?,?,?,?,?)", 
                         log_id, pos, url_id, is_click, host_id)
    for url, (pos, url_id, host_id, t_start, t_end) in click_dict.items():
        cursor.execute(f"insert into {click_tbl} (session_id, pos, url_id, host_id, t_start, t_end) \
                         values(?,?,?,?,?,?)", 
                         log_id, pos, url_id, host_id, t_start, t_end)
    return True

In [47]:
def process_file_by_query_thread(filename, 
                                 session_tbl='dbo.log_session_tmp', 
                                 show_tbl='dbo.log_show_tmp',
                                 click_tbl='dbo.log_click_tmp',
                                 check_session_tbl=True):
    try:
        process_file_by_query(filename, file_filter_dict[filename], 
                              session_tbl=session_tbl, 
                              show_tbl=show_tbl,
                              click_tbl=click_tbl,
                              check_session_tbl=check_session_tbl)
    except:
        print(filename)
    # счетчик должен атомарно обновиться
    with lock:
        pbar.update(1)

In [48]:
def process_file_by_query(filename, log_id_start_dict, 
                          session_tbl='dbo.log_session_tmp', 
                          show_tbl='dbo.log_show_tmp',
                          click_tbl='dbo.log_click_tmp', 
                          check_session_tbl=True, debug=False):
    if file_dict[filename]:
        return
    log_id_file = log_ids[filename][0]
    with bz2.open(os.path.join("2017", filename), "rb") as f:
        lines = f.readlines()
    
    log_id = log_id_file
    prev_query = ''
    log_id_start_list = sorted(log_id_start_dict.keys())
    log_id_start_idx = 0
    log_id_q = log_id_start_list[log_id_start_idx]
    q_dict = log_id_start_dict[log_id_q]
    with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
        for line_ind, line in enumerate(lines):
            if log_id < log_id_q:
                log_id = log_id + 1
                continue
            try:
                line = line.decode('utf-8').rstrip('\n')
                query = line.split("@")[0]
                if q_dict['search_query'] != query:
                    if log_id_start_idx + 1 < len(log_id_start_list):
                        log_id_start_idx = log_id_start_idx + 1
                        log_id_q = log_id_start_list[log_id_start_idx]
                        if log_id == log_id_q:
                            q_dict = log_id_start_dict[log_id_q]
                            #host_list = list(map(lambda x: get_host(x), search_urls))
                            #search_host_set = set(host_list)
                        else:
                            log_id = log_id + 1
                            continue
                    else:
                        break
                    
                process_log_line(cursor, log_id, q_dict['search_query_id'], line, 
                                 url_set=set(q_dict['urls']), host_set=None,
                                 session_tbl=session_tbl, 
                                 show_tbl=show_tbl,
                                 click_tbl=click_tbl, 
                                 check_session_tbl=check_session_tbl)
                
                if line_ind % 10_000 == 0:
                    conn.commit()
            except:
                print(filename, log_id, q_dict['search_query_id'])
                if debug:
                    return line, log_id, q_dict['search_query_id']
            log_id = log_id + 1
        conn.commit()
    file_dict[filename] = True

SET ANSI_NULLS ON  
GO  

SET QUOTED_IDENTIFIER ON  
GO  

CREATE TABLE [dbo].[log_session_tmp](  
	[id] [int] NOT NULL,  
	[query] [nvarchar](max) NULL,  
	[geo_id] [int] NOT NULL,  
	[query_id] [int] NULL,  
	[t_start] [bigint] NULL,  
	[t_end] [bigint] NULL,  
	[fclick_pos] [tinyint] NULL,  
	[lclick_pos] [tinyint] NULL,  
	[filename] [varchar](200) NULL,  
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]  
GO  

CREATE TABLE [dbo].[log_show_tmp](  
	[session_id] [int] NOT NULL,  
	[pos] [tinyint] NOT NULL,  
	[url_id] [int] NULL,  
	[is_click] [bit] NULL,  
	[host_id] [int] NULL,  
	[cm_is_skip] [bit] NULL,  
	[cm_is_not_seen] [bit] NULL  
) ON [PRIMARY]  
GO  

CREATE TABLE [dbo].[log_click_tmp](  
	[session_id] [int] NOT NULL,  
	[pos] [tinyint] NOT NULL,  
	[url_id] [int] NULL,  
	[t_start] [bigint] NULL,  
	[t_end] [bigint] NULL,  
	[host_id] [int] NULL  
) ON [PRIMARY]  
GO  

In [43]:
def process_file_by_url_thread(filename):
    try:
        process_file_by_url(filename)
    except:
        print(filename)
    # счетчик должен атомарно обновиться
    with lock:
        pbar.update(1)

In [42]:
def process_file_by_url(filename, log_id_start=0,
                        session_tbl='dbo.log_session_tmp', 
                        show_tbl='dbo.log_show_tmp',
                        click_tbl='dbo.log_click_tmp', 
                        check_session_tbl=True, debug=False):
    if file_dict[filename]:
        return
    log_id_file = log_ids[filename][0]
    with bz2.open(os.path.join("2017", filename), "rb") as f:
        lines = f.readlines()
    
    log_id = log_id_file + log_id_start
    prev_query = ''
    with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
        for line_ind, line in tqdm(enumerate(lines[log_id_start:]), total=len(lines[log_id_start:])):
            try:
                line = line.decode('utf-8').rstrip('\n')
                query = line.split("@")[0]
                if prev_query != query:
                    cursor.execute(f"select top 1 [id] FROM [ir].[dbo].[log_query] where [log_id_start] = {log_id}")
                    row = cursor.fetchone()
                    query_id = row[0]
                    prev_query = query
                    
                process_log_line(log_id, query_id, line, 
                                 url_set=url_set, host_set=None,
                                 session_tbl=session_tbl, 
                                 show_tbl=show_tbl,
                                 click_tbl=click_tbl, 
                                 check_session_tbl=check_session_tbl)

                if line_ind % 10_000 == 0:
                    conn.commit()
            except:
                print(filename, log_id, query_id)
                if debug:
                    return line, log_id, query_id
            log_id = log_id + 1
        conn.commit()
    file_dict[filename] = True

In [21]:
click_data_df[click_data_df.sim_q >= 0.5].shape

(54539, 7)

In [22]:
click_data_sim_q_df = click_data_df[click_data_df.sim_q >= 0.3]

In [23]:
files_unique = np.unique(click_data_sim_q_df['filename'], return_counts=False)
from collections import defaultdict
file_filter_dict = defaultdict(dict)

In [24]:
qids_unique = np.unique(train_df['query_id'], return_counts=False)
for qid in tqdm(qids_unique, total=len(qids_unique)):
    dids = train_df[train_df.query_id == qid].doc_id.values
    urls = url_data_df.loc[url_data_df['doc_id'].isin(dids)].doc_url.values
    click_data_q_df = click_data_sim_q_df[click_data_sim_q_df.query_id == qid]
    for _, row in click_data_q_df.iterrows():
        file_filter_dict[row.filename][row.log_id_start] = {'session_query_id': row.query_id, 
                                                            'search_query_id': row.c_query_id,
                                                            'search_query'  : row.c_query, 
                                                            'urls': urls}

In [25]:
qids_unique = np.unique(test_df['query_id'], return_counts=False)
for qid in tqdm(qids_unique, total=len(qids_unique)):
    dids = test_df[test_df.query_id == qid].doc_id.values
    urls = url_data_df.loc[url_data_df['doc_id'].isin(dids)].doc_url.values
    click_data_q_df = click_data_sim_q_df[click_data_sim_q_df.query_id == qid]
    for _, row in click_data_q_df.iterrows():
        file_filter_dict[row.filename][row.log_id_start] = {'session_query_id': row.query_id, 
                                                            'search_query_id': row.c_query_id,
                                                            'search_query'  : row.c_query, 
                                                            'urls': urls}

In [26]:
file_filter_dict = {k: v for k, v in file_filter_dict.items()}
file_dict = {k: False for k in file_filter_dict.keys()}
file_list = list(file_filter_dict.keys())

In [55]:
with open('file_errors.txt', 'r') as fin:
    file_errors = fin.readlines()
file_errors = list(set(file_errors))
file_errors = [f.rstrip('\n') for f in file_errors]

USE [ir]  
GO  

SET ANSI_NULLS ON  
GO  

SET QUOTED_IDENTIFIER ON  
GO  

CREATE TABLE [log].[session](  
	[id] [int] NOT NULL,  
	[query] [nvarchar](max) NULL,  
	[geo_id] [int] NOT NULL,  
	[query_id] [int] NULL,  
	[t_start] [bigint] NULL,  
	[t_end] [bigint] NULL,  
	[fclick_pos] [tinyint] NULL,  
	[lclick_pos] [tinyint] NULL,  
	[filename] [varchar](200) NULL  
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]  
GO  

CREATE TABLE [log].[show](  
	[session_id] [int] NOT NULL,  
	[pos] [tinyint] NOT NULL,  
	[url_id] [int] NULL,  
	[is_click] [bit] NULL,  
	[host_id] [int] NULL,  
	[cm_is_skip] [bit] NULL,  
	[cm_is_not_seen] [bit] NULL  
) ON [PRIMARY]  
GO  

CREATE TABLE [log].[click](  
	[session_id] [int] NOT NULL,  
	[pos] [tinyint] NOT NULL,  
	[url_id] [int] NULL,  
	[t_start] [bigint] NULL,  
	[t_end] [bigint] NULL,  
	[host_id] [int] NULL  
) ON [PRIMARY]  
GO  

In [27]:
from multiprocessing.dummy import Pool, Lock, Queue
with Pool(processes=4) as pool, tqdm(total=len(file_list)) as pbar:
    lock = pbar.get_lock()
    pool.map(process_file_by_query_thread, file_list)

part-m-00525.bz2 145094446 26675480
part-m-00525.bz2 145094449 26675480
part-m-00752.bz2 207686876 35419716
part-m-00752.bz2 207686877 35419716
part-m-00752.bz2 207686878 35419716
part-m-00752.bz2 207686880 35419716
part-m-00752.bz2 207686881 35419716
part-m-00752.bz2 207686882 35419716
part-m-01010.bz2 310171520 39074073
part-m-01010.bz2 310171522 39074073
part-m-01010.bz2 310309503 39402974
part-m-01010.bz2 310309506 39402974
part-m-00526.bz2 145549000 27305935
part-m-00526.bz2 145549002 27305935
part-m-00526.bz2 145745658 27680617
part-m-00526.bz2 145745660 27680617
part-m-00527.bz2 146006106 28180752
part-m-00527.bz2 146006108 28180752
part-m-00527.bz2 146093015 28391761
part-m-00527.bz2 146093016 28391761
part-m-00527.bz2 146093018 28391761
part-m-00527.bz2 146093019 28391761
part-m-00527.bz2 146098227 28405015
part-m-00527.bz2 146098228 28405015
part-m-00527.bz2 146098229 28405015
part-m-00527.bz2 146098230 28405015
part-m-00527.bz2 146098231 28405015
part-m-00527.bz2 146098232 2

part-m-00768.bz2 223202187 56903575
part-m-00768.bz2 223202188 56903575
part-m-00778.bz2 229607387 68055482
part-m-00778.bz2 229607388 68055482
part-m-00569.bz2 158357597 18276095
part-m-00569.bz2 158357599 18276095
part-m-00604.bz2 164184636 29118606
part-m-00604.bz2 164184637 29118606
part-m-00604.bz2 164184638 29118606
part-m-00604.bz2 164184639 29118606
part-m-00604.bz2 164184640 29118606
part-m-00604.bz2 164184641 29118606
part-m-00604.bz2 164184642 29118606
part-m-00604.bz2 164184643 29118606
part-m-00604.bz2 164184644 29118606
part-m-00604.bz2 164184645 29118606
part-m-00604.bz2 164184647 29118606
part-m-00604.bz2 164184648 29118606
part-m-00604.bz2 164184649 29118606
part-m-00604.bz2 164184650 29118606
part-m-00604.bz2 164184651 29118606
part-m-00604.bz2 164184652 29118606
part-m-00604.bz2 164184653 29118606
part-m-00604.bz2 164184654 29118606
part-m-00604.bz2 164184655 29118606
part-m-00604.bz2 164184656 29118606
part-m-00604.bz2 164184657 29118606
part-m-00604.bz2 164184658 2

part-m-00764.bz2 218985044 55365758
part-m-00764.bz2 218985045 55365758
part-m-00764.bz2 218985046 55365758
part-m-00764.bz2 218985047 55365758
part-m-00764.bz2 218985048 55365758
part-m-00764.bz2 218985049 55365758
part-m-00764.bz2 218985050 55365758
part-m-00764.bz2 218985051 55365758
part-m-00764.bz2 218985052 55365758
part-m-00764.bz2 218985054 55365758
part-m-00764.bz2 218985055 55365758
part-m-00764.bz2 218985056 55365758
part-m-00764.bz2 218985057 55365758
part-m-00764.bz2 218985058 55365758
part-m-00764.bz2 218985059 55365758
part-m-00764.bz2 218985060 55365758
part-m-00764.bz2 218985061 55365758
part-m-00764.bz2 218985062 55365758
part-m-00764.bz2 218985063 55365758
part-m-00764.bz2 218985064 55365758
part-m-00764.bz2 218985065 55365758
part-m-00764.bz2 218985066 55365758
part-m-00764.bz2 218985067 55365758
part-m-00764.bz2 218985068 55365758
part-m-00764.bz2 218985069 55365758
part-m-00764.bz2 218985070 55365758
part-m-00764.bz2 218985071 55365758
part-m-00764.bz2 218985072 5

part-m-00762.bz2 217517276 52799259
part-m-00762.bz2 217517277 52799259
part-m-00762.bz2 217517278 52799259
part-m-00762.bz2 217517279 52799259
part-m-00762.bz2 217517280 52799259
part-m-00762.bz2 217517281 52799259
part-m-00762.bz2 217517282 52799259
part-m-00762.bz2 217517283 52799259
part-m-00762.bz2 217517284 52799259
part-m-00762.bz2 217517285 52799259
part-m-00762.bz2 217517286 52799259
part-m-00762.bz2 217517287 52799259
part-m-00762.bz2 217517288 52799259
part-m-00762.bz2 217517289 52799259
part-m-00762.bz2 217517290 52799259
part-m-00762.bz2 217517291 52799259
part-m-00762.bz2 217517292 52799259
part-m-00762.bz2 217517293 52799259
part-m-00762.bz2 217517294 52799259
part-m-00762.bz2 217517295 52799259
part-m-00762.bz2 217517296 52799259
part-m-00762.bz2 217517297 52799259
part-m-00762.bz2 217517298 52799259
part-m-00762.bz2 217517299 52799259
part-m-00762.bz2 217517300 52799259
part-m-00762.bz2 217517301 52799259
part-m-00762.bz2 217517302 52799259
part-m-00762.bz2 217517303 5

part-m-00762.bz2 217517527 52799259
part-m-00762.bz2 217517528 52799259
part-m-00762.bz2 217517529 52799259
part-m-00762.bz2 217517530 52799259
part-m-00762.bz2 217517531 52799259
part-m-00762.bz2 217517532 52799259
part-m-00762.bz2 217517533 52799259
part-m-00762.bz2 217517534 52799259
part-m-00762.bz2 217517535 52799259
part-m-00762.bz2 217517536 52799259
part-m-00762.bz2 217517537 52799259
part-m-00762.bz2 217517538 52799259
part-m-00762.bz2 217517539 52799259
part-m-00762.bz2 217517540 52799259
part-m-00762.bz2 217517541 52799259
part-m-00762.bz2 217517542 52799259
part-m-00762.bz2 217517543 52799259
part-m-00762.bz2 217517544 52799259
part-m-00762.bz2 217517545 52799259
part-m-00762.bz2 217517546 52799259
part-m-00762.bz2 217517547 52799259
part-m-00762.bz2 217517548 52799259
part-m-00762.bz2 217517549 52799259
part-m-00762.bz2 217517550 52799259
part-m-00762.bz2 217517551 52799259
part-m-00762.bz2 217517552 52799259
part-m-00762.bz2 217517553 52799259
part-m-00762.bz2 217517554 5

part-m-00425.bz2 113168983 7961257
part-m-00425.bz2 113168984 7961257
part-m-00425.bz2 113168989 7961257
part-m-00425.bz2 113168990 7961257
part-m-00788.bz2 238075493 29469232
part-m-00788.bz2 238075494 29469232
part-m-00953.bz2 294268958 40853236
part-m-00953.bz2 294268959 40853236
part-m-00953.bz2 294268960 40853236
part-m-00953.bz2 294268961 40853236
part-m-00769.bz2 223955814 58808632
part-m-00769.bz2 223955815 58808632
part-m-00755.bz2 209529563 40048015
part-m-00755.bz2 209529565 40048015
part-m-01573.bz2 491507422 70256423
part-m-01573.bz2 491507424 70256423
part-m-01573.bz2 491570764 70290068
part-m-01573.bz2 491570766 70290068
part-m-01572.bz2 491474724 70185960
part-m-01572.bz2 491474725 70185960
part-m-00353.bz2 98785477 3152364
part-m-00353.bz2 98785478 3152364
part-m-00353.bz2 98785480 3152364
part-m-00353.bz2 98785482 3152364
part-m-00954.bz2 295243287 41321038
part-m-00954.bz2 295243288 41321038
part-m-00954.bz2 295243289 41321038
part-m-00954.bz2 295243294 41321038
part

USE ir;  
GO  
ALTER TABLE dbo.log_session  
ADD CONSTRAINT PK_log_session_id PRIMARY KEY CLUSTERED (id);  

ALTER TABLE dbo.log_session  
ADD CONSTRAINT FK_session_query FOREIGN KEY ([query_id])    
REFERENCES dbo.log_query ([id]);  

USE ir;  
GO
IF EXISTS (SELECT name FROM sys.indexes  
            WHERE name = N'ix_log_session_query_id')   
    DROP INDEX ix_log_session_query_id ON dbo.log_session;   
GO 
CREATE NONCLUSTERED INDEX ix_log_session_query_id   
    ON dbo.log_session (query_id);   
GO  

USE ir;  
GO  
IF EXISTS (SELECT name FROM sys.indexes  
WHERE name = N'ix_log_session_filename')  
DROP INDEX ix_log_session_filename ON dbo.log_session;  
GO  
CREATE NONCLUSTERED INDEX ix_log_session_filename  
ON dbo.log_session ([filename]);  
GO  

**log_click**

ALTER TABLE dbo.log_click  
ADD CONSTRAINT FK_click_session FOREIGN KEY ([session_id])  
REFERENCES dbo.log_session ([id]);  

USE ir;  
GO  
IF EXISTS (SELECT name FROM sys.indexes  
		   WHERE name = N'ix_log_click_url_id')  
DROP INDEX ix_log_click_url_id ON dbo.log_click;  
GO  
CREATE NONCLUSTERED INDEX ix_log_click_url_id  
ON dbo.log_click (url_id);  
GO  

USE ir; 
GO  
IF EXISTS (SELECT name FROM sys.indexes  
		   WHERE name = N'ix_log_click_host_id')  
DROP INDEX ix_log_click_host_id ON dbo.log_click;  
GO  
CREATE NONCLUSTERED INDEX ix_log_click_host_id  
ON dbo.log_click ([host_id]);
GO  

USE ir;  
GO  
IF EXISTS (SELECT name FROM sys.indexes  
		   WHERE name = N'ix_log_click_session_id')  
DROP INDEX ix_log_click_session_id ON dbo.log_click;  
GO  
CREATE NONCLUSTERED INDEX ix_log_click_session_id  
ON dbo.log_click(session_id);  
GO  

**log_show**

ALTER TABLE dbo.log_show  
ADD CONSTRAINT FK_show_session FOREIGN KEY ([session_id])  
REFERENCES dbo.log_session ([id]);  

USE ir;  
GO  
IF EXISTS (SELECT name FROM sys.indexes  
		   WHERE name = N'ix_log_show_url_id')  
DROP INDEX ix_log_show_url_id ON dbo.log_show;  
GO  
CREATE NONCLUSTERED INDEX ix_log_show_url_id  
ON dbo.log_show (url_id);  
GO  

USE ir; 
GO  
IF EXISTS (SELECT name FROM sys.indexes  
		   WHERE name = N'ix_log_show_host_id')  
DROP INDEX ix_log_show_host_id ON dbo.log_show;  
GO  
CREATE NONCLUSTERED INDEX ix_log_show_host_id  
ON dbo.log_show ([host_id]);  
GO  

USE ir;  
GO  
IF EXISTS (SELECT name FROM sys.indexes  
		   WHERE name = N'ix_log_show_session_id')  
DROP INDEX ix_log_show_session_id ON dbo.log_show;  
GO  
CREATE NONCLUSTERED INDEX ix_log_show_session_id  
ON dbo.log_show (session_id);  
GO  

In [99]:
file_sizes = {filename: os.path.getsize(os.path.join("2017", filename)) / 1024 for filename in file_filter_dict.keys()}
file_sizes = dict(sorted(file_sizes.items(), key=lambda item: item[1]))

In [110]:
file_sizes[file_errors[9]]

7653.2255859375

In [57]:
filename = 'part-m-00930.bz2'
file_dict[filename] = False
process_file_by_query(filename, file_filter_dict[filename], 
                      session_tbl='dbo.log_session_tmp', 
                      show_tbl='dbo.log_show_tmp',
                      click_tbl='dbo.log_click_tmp', 
                      check_session_tbl=True, debug=True)

In [43]:
log_id = 145094446
search_query_id = 26675480

In [44]:
with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
    process_log_line(cursor, log_id, search_query_id, line,
                     url_set=None,
                     host_set=None,
                     session_tbl='dbo.log_session_tmp', 
                     show_tbl='dbo.log_show_tmp',
                     click_tbl='dbo.log_click_tmp', 
                     check_session_tbl=True)
    conn.commit()

In [45]:
parse_log_line(line)

(True,
 'где оплатить налог на имущество в казани',
 '188',
 {'sovetclub.ru/kak-oplatit-nalog-na-imushhestvo': (1, 472538, 107111, 1),
  'bolshoyvopros.ru/questions/1222729-kakim-obrazom-i-gde-mozhno-oplatit-nalogi-bez-komissii.html': (2,
   None,
   16317,
   0),
  'rg.ru/2013/10/15/umushestvo.html': (3, None, 94510, 0),
  'avto-flot.ru/blog/gde-oplatit-transportnyiy-nalog-byistro-i-bez-komissii.html': (4,
   None,
   11163,
   0),
  'buhonline24.ru/nalogi/na-imushhestvo/oplata-naloga-cherez-internet.html': (5,
   183777,
   17390,
   1),
  'kzn.ru/news/55916-kazantsy-dolzhny-oplatit-imuschestvennye-nalogi-do-1-dekabrya': (6,
   None,
   59029,
   0),
  'oplatagosuslug.ru/read/important/nalogi-online': (7, None, 81304, 0),
  'service.nalog.ru/payment/tax-fl.html': (8, None, 101929, 0),
  'decoxl.ru/6154': (9, None, None, 1),
  'a-3.ru/pay_taxes': (10, None, 2781, 0)},
 {'sovetclub.ru/kak-oplatit-nalog-na-imushhestvo': (1,
   472538,
   107111,
   1509343534,
   1509343587),
  'buhonli

insert into [ir].[dbo].[log_session]  
([id] ,[query] ,[geo_id] ,[query_id] ,[t_start] ,[t_end] ,[fclick_pos] ,[lclick_pos] ,[filename])  
select [id]  
      ,[query]  
      ,[geo_id]  
      ,[query_id]  
      ,[t_start]  
      ,[t_end]  
      ,[fclick_pos]  
      ,[lclick_pos]  
      ,[filename]  
  FROM [ir].[dbo].[log_session_tmp]  
  
insert into [ir].[dbo].[log_click]  
([session_id] ,[pos] ,[url_id] ,[t_start] ,[t_end] ,[host_id])  
select [session_id] ,[pos] ,[url_id] ,[t_start] ,[t_end] ,[host_id]  
FROM [ir].[dbo].[log_click_tmp]  

insert into [ir].[dbo].[log_show]  
([session_id] ,[pos] ,[url_id],[is_click] ,[host_id] ,[cm_is_skip] ,[cm_is_not_seen])  
select [session_id] ,[pos] ,[url_id],[is_click] ,[host_id] ,[cm_is_skip] ,[cm_is_not_seen]  
FROM [ir].[dbo].[log_show_tmp]  

USE ir  
ALTER DATABASE ir SET RECOVERY SIMPLE  
go   
DBCC SHRINKFILE (ir_log, 1);  

truncate table dbo.log_session_tmp  
truncate table dbo.log_click_tmp  
truncate table dbo.log_show_tmp  

### Расчет поведенческих факторов

In [86]:
limit = 10
ranker = 'sph04'
fileds = ['id', 'query', 'filename', 'log_id_start']
index = 'log_query'
query = helpers.cut_query('а не спеть ли мне песню о любви')
rows = helpers.select_from_sphinx_wrapper(query, index, fileds, limit, ranker)

In [82]:
[row[3] for row in rows]

[90077298,
 90078344,
 90078342,
 6789519,
 6864512,
 6870702,
 6893074,
 90078555,
 90078976,
 6516270]

### Отбор файлов

In [79]:
import pandas as pd

In [80]:
queries_df = pd.read_csv('corrected_queries.tsv', sep='\t')

In [88]:
limit = 10
ranker = 'sph04'
fileds = ['id', 'query', 'filename', 'log_id_start', 'weight()']
index = 'log_query'

qid_list = []
c_qid_list = []
c_query_list = []
c_filename_list = []
c_log_id_start_list = []
c_query_weight_list = []

with helpers.get_sphinx_conn() as sphinx_conn:
    for _, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
        query = helpers.cut_query(row.corrected_query)
        docs = helpers.select_from_sphinx(sphinx_conn, query, index, fileds, limit, ranker)
        for doc in docs:
            qid_list.append(row.id)
            c_qid_list.append(doc[0])
            c_query_list.append(doc[1])
            c_filename_list.append(doc[2])
            c_log_id_start_list.append(doc[3])
            c_query_weight_list.append(doc[4])

In [89]:
click_data_dict = {'query_id': qid_list, 'c_query_id': c_qid_list, 
                   'c_query': c_query_list, 'filename': c_filename_list, 
                   'log_id_start': c_log_id_start_list, 'c_query_weight': c_query_weight_list}
click_data_df = pd.DataFrame(data = click_data_dict)

In [91]:
helpers.save_obj(click_data_df, 'click_data_df')

In [83]:
click_data_df.to_csv('click_data_df.csv', index=False, sep='\t')

### Парсинг файлов для расчета click models

In [49]:
class ClickDataParser:
    @staticmethod
    def parse_filter(sessions_filename, log_id_start_dict, sessions_max=None):
        """
        Parses search sessions, formatted according to:
        Query Text @ Query Geo <tab> List of shown urls <tab> List of clicked urls <tab>
        :param sessions_filename: The name of the file with search sessions.
        :param sessions_max: The maximum number of search sessions to return.
        If not set, all search sessions are parsed and returned.
        :returns: A list of parsed search sessions, wrapped into SearchSession objects.
        """
        log_id_file = log_ids[sessions_filename][0]
        with bz2.open(os.path.join("2017", sessions_filename), "rb") as f:
            lines = f.readlines()

        sessions = []

        log_id = log_id_file
        log_id_start_list = sorted(log_id_start_dict.keys())
        log_id_idx = 0
        log_id_q = log_id_start_list[log_id_idx]
        for line in lines:
            if log_id < log_id_q:
                log_id = log_id + 1
                continue
            if sessions_max and len(sessions) >= sessions_max:
                break
            if log_id == log_id_q:
                q_dict = log_id_start_dict[log_id_q]

            line = line.decode('utf-8').rstrip('\n')
            query = line.split("@")[0]

            if q_dict['search_query'] != query:
                if log_id_start_idx + 1 < len(log_id_start_list):
                    log_id_idx = log_id_idx + 1
                    log_id_q = log_id_start_list[log_id_idx]
                    if log_id == log_id_q:
                        q_dict = log_id_start_dict[log_id_q]
                    else:
                        log_id = log_id + 1
                        continue
                else:
                    break

            line = line[len(query) + 1:]
            data = line.split('\t')

            task = log_id + log_id_file
            results = dict(map(lambda x: (x.split("://")[-1], None) if x.find('://') > 0 else (x, None), 
                               data[1].split(',')))

            if not (set(q_dict['urls']) & set(results)):
                log_id = log_id + 1
                continue

            click_results = dict(map(lambda x: (x.split("://")[-1], None) if x.find('://') > 0 else (x, None), 
                                     data[2].split(',')))

            session = TaskCentricSearchSession(task, q_dict['session_query_id'])

            for result in results:
                click = 1 if result in click_results else 0
                result = SearchResult(result, click)
                session.web_results.append(result)

            sessions.append(session)
            log_id = log_id + 1            
        return sessions

In [64]:
click_data_df = click_data_df.sort_values(by=['query_id', 'filename', 'log_id_start'])

In [92]:
files_unique = np.unique(click_data_df['filename'], return_counts=False)

In [93]:
len(files_unique)

1452

In [95]:
url_data_df = pd.read_csv('url.data', header=None, sep='\t')
url_data_df.rename(columns={0: "doc_id", 1: "url_id"}, inplace=True)

In [97]:
train_df = pd.read_csv('mark_data_df_use_qa.csv', sep='\t')
test_df = pd.read_csv('sample_data_df_use_qa.csv', sep='\t')

In [103]:
from collections import defaultdict

In [179]:
file_filter_dict = defaultdict(dict)

In [180]:
qids_unique = np.unique(train_df['query_id'], return_counts=False)
for qid in tqdm(qids_unique, total=len(qids_unique)):
    dids = train_df[train_df.query_id == qid].doc_id.values
    urls = url_data_df.loc[url_data_df['doc_id'].isin(dids)].url_id.values
    click_data_q_df = click_data_df[click_data_df.query_id == qid]
    for _, row in click_data_q_df.iterrows():
        file_filter_dict[row.filename][row.log_id_start] = {'session_query_id': row.query_id, 
                                                            'search_query_id': row.c_query_id,
                                                            'search_query'  : row.c_query, 
                                                            'urls': urls}

In [181]:
qids_unique = np.unique(test_df['query_id'], return_counts=False)
for qid in tqdm(qids_unique, total=len(qids_unique)):
    dids = test_df[test_df.query_id == qid].doc_id.values
    urls = url_data_df.loc[url_data_df['doc_id'].isin(dids)].url_id.values
    click_data_q_df = click_data_df[click_data_df.query_id == qid]
    for _, row in click_data_q_df.iterrows():
        file_filter_dict[row.filename][row.log_id_start] = {'session_query_id': row.query_id, 
                                                            'search_query_id': row.c_query_id,
                                                            'search_query'  : row.c_query, 
                                                            'urls': urls}

In [109]:
from pyclick.click_models.task_centric.TaskCentricSearchSession import TaskCentricSearchSession
from pyclick.search_session.SearchResult import SearchResult

In [182]:
search_sessions_path = files_unique[0]

In [183]:
f_search_sessions = ClickDataParser().parse_filter(search_sessions_path, file_filter_dict[search_sessions_path])

In [185]:
search_sessions = []
for search_sessions_path in tqdm(files_unique):
    f_search_sessions = ClickDataParser().parse_filter(search_sessions_path, file_filter_dict[search_sessions_path])
    search_sessions.extend(f_search_sessions)

In [186]:
len(search_sessions)

3542389

In [187]:
helpers.save_obj(search_sessions, 'search_sessions')

In [110]:
from __future__ import print_function

import sys

import time

from pyclick.click_models.Evaluation import LogLikelihood, Perplexity
from pyclick.click_models.UBM import UBM
from pyclick.click_models.DBN import DBN
from pyclick.click_models.SDBN import SDBN
from pyclick.click_models.DCM import DCM
from pyclick.click_models.CCM import CCM
from pyclick.click_models.CTR import DCTR, RCTR, GCTR
from pyclick.click_models.CM import CM
from pyclick.click_models.PBM import PBM
from pyclick.utils.Utils import Utils

In [146]:
def predict_relevance(click_model, query, search_result):
    return click_model.params[click_model.param_names.attr].get(query, search_result).value()

In [283]:
def cut_session_to_10(search_sessions):
    search_sessions_cut = []
    for s in tqdm(search_sessions, total=len(search_sessions)):
        s_cut = TaskCentricSearchSession(s.task, s.query)
        s_cut.web_results = s.web_results[:10]
        search_sessions_cut.append(s_cut)
    return search_sessions_cut

In [145]:
def get_rels(df, click_model, predict='custom'):
    rels = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        if predict == 'custom':
            rel = predict_relevance(click_model, row.query_id, row.doc_url)
        else:
            rel = click_model.predict_relevance(row.query_id, row.doc_url)
        rels.append(rel)
    return rels

In [270]:
def get_urls(df):
    urls = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        url = url_data_df[url_data_df.doc_id == row.doc_id].url_id.values[0]
        urls.append(url)
    return urls

In [271]:
train_urls = get_urls(train_df)
train_df['doc_url'] = train_urls

In [272]:
test_urls = get_urls(test_df)
test_df['doc_url'] = test_urls

### Расчет статистики по кликовым данным

In [122]:
import math
from pyclick.search_session.SearchSession import SearchSession

In [81]:
alpha = np.array([0.41, 0.16, 0.105, 0.08, 0.06, 0.05, 0.35, 0.3, 0.25, 0.2])

In [571]:
def get_session_by_qid(qid, conn):
    with conn.cursor() as cursor:
        cursor.execute(f'select s.[id], sh.[pos], sh.[url_id], sh.[is_click] \
                        from [ir].[dbo].[log_session] s inner join [ir].[dbo].[log_show] sh on s.id = sh.session_id \
                        where s.query_id = {qid} \
                        order by s.id, sh.pos')
        rows = cursor.fetchall()
    return pd.DataFrame.from_records(rows, columns=['session_id', 'pos', 'url_id', 'is_click'])

In [343]:
from pyclick.click_models.Inference import EMInference, MLEInference
click_models = {'cm_rel': (CM(), False, True), 
                'sdbn_rel': (SDBN(), False, False), 
                'dctr_rel': (DCTR(), False, False),
                #'ubm_rel': (UBM(EMInference(iter_num=1)), True, False),
                #'dbn_rel': (DBN(EMInference(iter_num=1)), False, False)
               }

In [574]:
def get_search_sessions(q_df, c_query_id, qid, smooth=True):
    s_ids, cnt_arr = np.unique(q_df.session_id, return_counts=True)
    search_sessions = []
    search_sessions_cut = []
    start_id = 0
    for i, s_id in enumerate(s_ids):
        end_id = start_id + cnt_arr[i]
        q_s_df = q_df.iloc[start_id:end_id]
        q_s_df.url_id = q_s_df.url_id.fillna(-1).values
        if smooth:
            s_cut = SearchSession(c_query_id)
            s = SearchSession(c_query_id)
        else:
            s_cut = SearchSession(qid)
            s = SearchSession(qid)
        for _, row in q_s_df.iterrows():
            result = SearchResult(int(row.url_id) if row.url_id != -1 else None, row.is_click)
            s.web_results.append(result)
        s_cut.web_results = s.web_results[:10]
        
        search_sessions.append(s)
        search_sessions_cut.append(s_cut)
        
        start_id = end_id
    return search_sessions, search_sessions_cut

In [575]:
def train_and_get_cm_by_qid(search_sessions, search_sessions_cut):
    models = {'cm_rel': (CM(), False, True), 
              'sdbn_rel': (SDBN(), False, False), 
              'dctr_rel': (DCTR(), False, False),
              #'ubm_rel': (UBM(EMInference(iter_num=1)), True, False),
              #'dbn_rel': (DBN(EMInference(iter_num=1)), False, False)
             }
    for k, v in models.items():
        model = v[0]
        need_cut = v[1]
        model.train(search_sessions) if not need_cut else model.train(search_sessions_cut)
    return models

In [581]:
def get_cm_stat_by_q(conn, qid, qids_aux, sim_aux, dids):
    if len(sim_aux) == 0:
        return {k: [0] * len(dids) for k in click_models.keys()}
    
    click_features_dict = {}
    search_sessions = [] 
    search_sessions_cut = []
    for c_query_id in qids_aux:
        q_df = get_session_by_qid(c_query_id, conn)
        s, s_cut = get_search_sessions(q_df, c_query_id, qid, smooth=False)
        search_sessions.extend(s)
        search_sessions_cut.extend(s_cut)
    models = train_and_get_cm_by_qid(search_sessions, search_sessions_cut)
    for k, v in models.items():
        model = v[0]
        predict_relevance_custom = v[2]
        rel_list = list(map(lambda did: get_rel(model, qid, did, predict_relevance_custom),
                            dids))
        click_features_dict[k] = rel_list
    return click_features_dict

In [590]:
def get_cm_stat_by_q_smooth(conn, qid, qids_aux, sim_aux, dids):
    if len(sim_aux) == 0:
        return {k: [0] * len(dids) for k in click_models.keys()}
    
    click_features_dict = {k: [] for k in click_models.keys()}
    for c_query_id in qids_aux:
        q_df = get_session_by_qid(c_query_id, conn)
        if q_df.shape[0] > 0:
            search_sessions, search_sessions_cut = get_search_sessions(q_df, c_query_id, qid, smooth=True)
            models = train_and_get_cm_by_qid(search_sessions, search_sessions_cut)

            for k, v in models.items():
                model = v[0]
                predict_relevance_custom = v[2]
                rel_list = list(map(lambda did: get_rel(model, c_query_id, did, predict_relevance_custom), 
                                    dids))
                click_features_dict[k].append(rel_list)
        else:
            for k in click_models.keys():
                click_features_dict[k].append([0] * len(dids))

    click_features_smooth_dict = {}
    for k in click_models.keys():
        cf_smooth = np.mean(np.array(click_features_dict[k]) * sim_aux[:,np.newaxis], axis=0)
        click_features_smooth_dict[k] = cf_smooth
    return click_features_smooth_dict

In [578]:
def get_rel(click_model, qid, did, predict_relevance_custom=False):
    if predict_relevance_custom:
        return predict_relevance(click_model, qid, did)
    else:
        return click_model.predict_relevance(qid, did)

In [416]:
len(np.unique(click_data_df[click_data_df.sim_q >= 0.8].query_id, return_counts=False))

5540

In [66]:
click_data_sim_q_df = click_data_df[click_data_df.sim_q >= 0.3]

In [591]:
qids_unique = np.unique(train_df['query_id'], return_counts=False)
click_features_dict = {k: [] for k in click_models.keys()}
all_dids = []
with helpers.get_sql_conn() as conn:
    for qid in tqdm(qids_unique):
        dids = train_df[train_df.query_id == qid].doc_id.values
        all_dids.extend(dids)
        aux_df = click_data_sim_q_df[click_data_sim_q_df.query_id == qid][['c_query_id', 'sim_q']]
        qids_aux = aux_df.c_query_id.values
        sim_aux = aux_df.sim_q.values
        
        cf_q = get_cm_stat_by_q_smooth(conn, qid, qids_aux, sim_aux, dids)
        for k in click_models.keys():
            click_features_dict[k].extend(cf_q[k])

In [592]:
data_dict = {'doc_id': all_dids}
data_dict.update(click_features_dict)
data_df = pd.DataFrame(data = data_dict)
data_df['query_id'] = train_df.query_id.values

In [ ]:
data_df.to_csv('train_cm_smooth.csv', index=False, sep='\t')

In [613]:
qids_unique = np.unique(test_df['query_id'], return_counts=False)
click_features_dict = {k: [] for k in click_models.keys()}
all_dids = []
with helpers.get_sql_conn() as conn:
    for qid in tqdm(qids_unique):
        dids = test_df[test_df.query_id == qid].doc_id.values
        all_dids.extend(dids)
        aux_df = click_data_sim_q_df[click_data_sim_q_df.query_id == qid][['c_query_id', 'sim_q']]
        qids_aux = aux_df.c_query_id.values
        sim_aux = aux_df.sim_q.values
        
        cf_q = get_cm_stat_by_q_smooth(conn, qid, qids_aux, sim_aux, dids)
        for k in click_models.keys():
            click_features_dict[k].extend(cf_q[k])

In [614]:
data_dict = {'doc_id': all_dids}
data_dict.update(click_features_dict)
data_df = pd.DataFrame(data = data_dict)
data_df['query_id'] = test_df.query_id.values

In [615]:
data_df.to_csv('test_cm_smooth.csv', index=False, sep='\t')

In [401]:
train_cm_df = pd.read_csv('train_cm_df.csv', sep='\t')

In [410]:
train_cm_df.sort_values(by=['query_id', 'doc_id'], inplace=True)

In [606]:
train_cm_df[train_cm_df.doc_id == 4515]

,query_id,doc_id,cm_rel,dctr_rel,gctr_rel,sdbn_rel,dcm_rel,ubm_rel,dbn_rel
168254,5344,4515,0.908571,0.787129,0.134955,0.75525,0.908571,0.856436,0.425592


In [611]:
data_df.shape, train_cm_df.shape

((199886, 5), (199886, 9))

update [log_show] set cm_is_not_seen = 1  
FROM [log_show] sh  
inner join [log_session] s on sh.session_id = s.id  
and is_click = 0 and pos > s.fclick_pos and pos > s.lclick_pos  
and url_id is not null  

update [log_show] set cm_is_skip = 1  
FROM [log_show] sh  
inner join [log_session] s on sh.session_id = s.id  
and is_click = 0 and (pos < s.fclick_pos or pos < s.lclick_pos)  
and url_id is not null  

In [ ]:
click_data_df[['query_id', 'c_query_id', 'sim_q']].to_csv('sim_q.csv', index=False, sep='\t')

USE ir;  
GO  
IF EXISTS (SELECT name FROM sys.indexes  
            WHERE name = N'ix_sim_q_query_id')   
    DROP INDEX ix_sim_q_query_id ON dbo.sim_q;   
GO  
CREATE NONCLUSTERED INDEX ix_sim_q_query_id   
    ON dbo.sim_q(query_id);   
GO  

IF EXISTS (SELECT name FROM sys.indexes  
            WHERE name = N'ix_sim_q_log_query_id')   
    DROP INDEX ix_sim_q_log_query_id ON dbo.sim_q;   
GO  
CREATE NONCLUSTERED INDEX ix_sim_q_log_query_id   
    ON dbo.sim_q(log_query_id);   
GO  

In [933]:
def get_click_show_stat(by, field):
    with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
        cursor.execute(f"select {','.join(by)}, count([session_id]) as 'shows',\
                         avg([pos]) as 'avg_pos',sum(cast(is_click as int)) as 'clicks', \
                         sum(cast([cm_is_skip] as int)) as 'skips',\
                         sum(cast([cm_is_not_seen] as int)) as 'not_seens' \
                         from [ir].[dbo].[log_show] sh \
                         where {field} is not null \
                         group by {','.join(by)}")
        rows = cursor.fetchall()
    return pd.DataFrame.from_records(rows, columns=by+['shows', 'avg_pos', 'clicks', 'skips', 'not_seens'])

In [946]:
def get_click_stat(by, field):
    with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
        cursor.execute(f"select {','.join(by)}, \
                         avg(isnull(c.[t_end], c.[t_start] + 5*60)  - c.[t_start]) as 'avg_time' \
                         from [ir].[dbo].[log_click] c \
                         where {field} is not null \
                         group by {','.join(by)}")
        rows1 = cursor.fetchall()
        
        cursor.execute(f"select {','.join(by)},pos, \
                         count(c.[session_id]) as 'pos_clicks' \
                         from [ir].[dbo].[log_click] c \
                         where {field} is not null \
                         group by {','.join(by)}, pos")
        rows2 = cursor.fetchall()
        
        cursor.execute(f"select {','.join(by)},count([session_id]) as 'f_clicks' \
                         from [log_show] sh \
                         inner join [log_session] s on sh.session_id = s.id \
                         where {field} is not null \
                         and pos = s.fclick_pos \
                         group by {','.join(by)}")
        rows3 = cursor.fetchall()
        
        cursor.execute(f"select {','.join(by)},count([session_id]) as 'l_clicks' \
                         from [log_show] sh \
                         inner join [log_session] s on sh.session_id = s.id \
                         where {field} is not null \
                         and pos = s.lclick_pos \
                         group by {','.join(by)}")
        rows4 = cursor.fetchall()
    return pd.DataFrame.from_records(rows1, columns=by+['avg_time']), \
           pd.DataFrame.from_records(rows2, columns=by+['pos', 'pos_clicks']), \
           pd.DataFrame.from_records(rows3, columns=by+['f_clicks']), \
           pd.DataFrame.from_records(rows4, columns=by+['l_clicks'])

In [948]:
def get_out10_stat(by, field):
    with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
        cursor.execute(f"select {','.join(by)}, count([session_id]) as 'shows' \
                         from [ir].[dbo].[log_show] sh \
                         where {field} is not null and pos > 10 \
                         group by {','.join(by)}")
        rows = cursor.fetchall()
    return pd.DataFrame.from_records(rows, columns=by+['out_top10_shows'])

In [950]:
def get_pos_click_stat(by, field):
    with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
        q = f"select {','.join(by)}, pos, sum(cast(is_click as int)) as 'pos_clicks', \
              count([session_id]) as 'pos_shows' \
              from [ir].[dbo].[log_show] sh \
              where {field} is not null \
              and pos <= 10 \
              group by {','.join(by)}, pos"
        cursor.execute(q)
        rows1 = cursor.fetchall()
        
        q = f"select {','.join(by)}, avg([pos]) as 'avg_click_pos' \
              from [ir].[dbo].[log_show] sh \
              where {field} is not null \
              and sh.is_click = 1 \
              group by {','.join(by)}"
        cursor.execute(q)
        rows2 = cursor.fetchall()
    return pd.DataFrame.from_records(rows1, columns=by+['pos', 'pos_clicks', 'pos_shows']), \
           pd.DataFrame.from_records(rows2, columns=by+['avg_click_pos'])

In [951]:
def get_stat_dict_by_url(field, field_name_df):
    click_stat_dict = {}
    click_show_stat = get_click_show_stat([field], field)
    click_avg_stat, click_pos_click_stat, f_click_stat, l_click_stat = get_click_stat([field], field)
    out10_stat = get_out10_stat([field], field)
    show_pos_click_stat, avg_click_pos_stat = get_pos_click_stat([field], field)
    
    avg_click_pos_dict = {row[field]: row['avg_click_pos'] for _, row in avg_click_pos_stat.iterrows()}
    
    f_click_dict = {row[field]: row['f_clicks'] for _, row in f_click_stat.iterrows()}
    l_click_dict = {row[field]: row['l_clicks'] for _, row in l_click_stat.iterrows()}
    
    show_pos_click_dict = {(row[field], row['pos']): {c: row[c] for c in show_pos_click_stat.columns[2:]} 
                           for _, row in show_pos_click_stat.iterrows()}
    
    click_pos_click_dict = {(row[field], row['pos']): {c: row[c] for c in click_pos_click_stat.columns[2:]} 
                            for _, row in click_pos_click_stat.iterrows()}
    
    click_show_dict = {row[field]: {c: row[c] for c in click_show_stat.columns[1:]} 
                       for _, row in click_show_stat.iterrows()}
    
    click_avg_dict = {row[field]: {c: row[c] for c in click_avg_stat.columns[1:]} 
                      for _, row in click_avg_stat.iterrows()}
    
    out10_dict = {row[field]: {c: row[c] for c in out10_stat.columns[1:]} 
                  for _, row in out10_stat.iterrows()}
    
    
    def_click_show = {c: 0 for c in click_show_stat.columns[1:]}
    def_click_avg = {c: 0 for c in click_avg_stat.columns[1:]}
    def_out10_stat = {c: 0 for c in out10_stat.columns[1:]}
    
    for _, row in tqdm(qdh_df.iterrows(), total=len(qdh_df)):
        key = row[field_name_df]
        if not key in click_stat_dict:
            click_stat_dict[key] = {}
        else:
            continue
        click_show_by_f = click_show_dict[key] if key in click_show_dict else def_click_show
        click_avg_by_f = click_avg_dict[key] if key in click_avg_dict else def_click_avg
        out10_stat_by_f = out10_dict[key] if key in out10_dict else def_out10_stat

        click_stat_dict[key]['avg_click_pos'] = avg_click_pos_dict[key] if key in avg_click_pos_dict else 0
        click_stat_dict[key]['avg_time'] = click_avg_by_f['avg_time']
        click_stat_dict[key]['avg_pos'] = click_show_by_f['avg_pos']
        
        out_top10_shows = out10_stat_by_f['out_top10_shows']
        clicks = click_show_by_f['clicks']
        avg_pos = int(click_stat_dict[key]['avg_pos'])
        skips = click_show_by_f['skips']
        not_seens = click_show_by_f['not_seens']
        shows = click_show_by_f['shows']
        f_clicks = f_click_dict[key] if key in f_click_dict else 0
        l_clicks = l_click_dict[key] if key in l_click_dict else 0
        
        for i in range(10):
            key_pos = (key, i + 1)
            show_pos_clicks = show_pos_click_dict[key_pos]['pos_clicks'] \
                              if key_pos in show_pos_click_dict \
                              else 0
            click_pos_clicks = click_pos_click_dict[key_pos]['pos_clicks'] \
                               if key_pos in click_pos_click_dict \
                               else 0
            pos_shows = show_pos_click_dict[key_pos]['pos_shows'] \
                        if key_pos in show_pos_click_dict \
                        else 0
            
            click_stat_dict[key][f"{i + 1}_show_pos_prob_click"] = show_pos_clicks / clicks if clicks > 0 else 0
            click_stat_dict[key][f"{i + 1}_click_pos_prob_click"] = click_pos_clicks / clicks if clicks > 0 else 0
            click_stat_dict[key][f"{i + 1}_ctr"] = show_pos_clicks / pos_shows if pos_shows > 0 else 0
        
        click_stat_dict[key]['first_click_prob'] = f_clicks / clicks if clicks > 0 else 0
        click_stat_dict[key]['last_click_prob'] = l_clicks / clicks if clicks > 0 else 0
        
        click_stat_dict[key]['first_ctr'] = f_clicks / shows if shows > 0 else 0
        click_stat_dict[key]['last_ctr'] = l_clicks / shows if shows > 0 else 0
        
        click_stat_dict[key]['ctr'] = clicks / shows if shows > 0 else 0
        click_stat_dict[key]['skip_proba'] = skips / shows if shows > 0 else 0
        click_stat_dict[key]['not_seen_proba'] = not_seens / shows if shows > 0 else 0

        a = alpha[avg_pos - 1] if avg_pos <= 10 else 0
        click_stat_dict[key]['pbm'] = clicks / (shows * a) if (shows * a) > 0 else 0

        click_stat_dict[key]['out10_proba'] = out_top10_shows / shows if shows > 0 else 0

        click_stat_dict[key]['log(1+clicks)'] = np.log(1 + clicks)
        click_stat_dict[key]['log(1+shows)'] = np.log(1 + shows)
    return click_stat_dict

In [813]:
tqdm.pandas()

In [675]:
qdh_df = pd.concat([train_df[['query_id', 'doc_id']], test_df[['query_id', 'doc_id']]], axis=0)

In [ ]:
tqdm.pandas()
qdh_df['host_id'] = qdh_df.progress_apply(lambda row: dh_map[row.doc_id], axis=1)

In [952]:
click_stat_h = get_stat_dict_by_url('host_id', 'host_id')

In [953]:
click_stat_d = get_stat_dict_by_url('url_id', 'doc_id')

In [847]:
def get_click_show_stat_by_q(field):
    with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
        cursor.execute(f"select sim.query_id,{field},count([session_id]) as 'shows',\
                        avg([pos]) as 'avg_pos',sum(cast(is_click as int)) as 'clicks' \
                        ,sum(cast([cm_is_skip] as int)) as 'skips',\
                        sum(cast([cm_is_not_seen] as int)) as 'not_seens' \
                        from [ir].[dbo].[log_show] sh \
                        inner join log_session s on sh.session_id = s.id \
                        inner join sim_q sim on sim.log_query_id = s.query_id \
                        where {field} is not null \
                        and sim.sim_q >= 0 \
                        group by sim.query_id, {field}")
        rows = cursor.fetchall()
    return pd.DataFrame.from_records(rows, columns=['query_id', field, 'shows', 'avg_pos',
                                                    'clicks', 'skips', 'not_seens'])

In [961]:
def get_click_stat_by_q(field):
    with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
        cursor.execute(f"select sim.query_id,{field},\
                         avg(isnull(c.[t_end], c.[t_start] + 5*60)  - c.[t_start]) as 'avg_time' \
                         from [ir].[dbo].[log_click] c \
                         inner join log_session s on c.session_id = s.id \
                         inner join sim_q sim on sim.log_query_id = s.query_id \
                         where {field} is not null \
                         and sim.sim_q >= 0 \
                         group by sim.query_id, {field}")
        rows1 = cursor.fetchall()
        
        cursor.execute(f"select sim.query_id,{field},pos, \
                         count(c.[session_id]) as 'pos_clicks' \
                         from [ir].[dbo].[log_click] c \
                         inner join log_session s on c.session_id = s.id \
                         inner join sim_q sim on sim.log_query_id = s.query_id \
                         where {field} is not null \
                         and sim.sim_q >= 0 \
                         group by sim.query_id, {field}, pos")
        rows2 = cursor.fetchall()
        
        cursor.execute(f"select sim.query_id,{field},count([session_id]) as 'f_clicks' \
                         from [log_show] sh \
                         inner join [log_session] s on sh.session_id = s.id \
                         inner join sim_q sim on sim.log_query_id = s.query_id \
                         where {field} is not null \
                         and sim.sim_q >= 0 \
                         and pos = s.fclick_pos \
                         group by sim.query_id, {field}")
        rows3 = cursor.fetchall()
        
        cursor.execute(f"select sim.query_id,{field},count([session_id]) as 'l_clicks' \
                         from [log_show] sh \
                         inner join [log_session] s on sh.session_id = s.id \
                         inner join sim_q sim on sim.log_query_id = s.query_id \
                         where {field} is not null \
                         and sim.sim_q >= 0 \
                         and pos = s.lclick_pos \
                         group by sim.query_id, {field}")
        rows4 = cursor.fetchall()
    return pd.DataFrame.from_records(rows1, columns=['query_id', field, 'avg_time']), \
           pd.DataFrame.from_records(rows2, columns=['query_id', field, 'pos', 'pos_clicks']), \
           pd.DataFrame.from_records(rows3, columns=['query_id', field, 'f_clicks']), \
           pd.DataFrame.from_records(rows4, columns=['query_id', field, 'l_clicks'])

In [849]:
def get_out10_stat_by_q(field):
    with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
        cursor.execute(f"select sim.query_id,{field}, count([session_id]) as 'shows' \
                         from [ir].[dbo].[log_show] sh \
                         inner join log_session s on sh.session_id = s.id \
                         inner join sim_q sim on sim.log_query_id = s.query_id \
                         where {field} is not null and pos > 10 \
                         and sim.sim_q >= 0 \
                         group by sim.query_id, {field}")
        rows = cursor.fetchall()
    return pd.DataFrame.from_records(rows, columns=['query_id', field, 'out_top10_shows'])

In [962]:
def get_pos_click_stat_by_q(field):
    with helpers.get_sql_conn() as conn, conn.cursor() as cursor:
        q = f"select sim.query_id,{field}, pos, sum(cast(is_click as int)) as 'pos_clicks', \
              count([session_id]) as 'pos_shows' \
              from [ir].[dbo].[log_show] sh \
              inner join log_session s on sh.session_id = s.id \
              inner join sim_q sim on sim.log_query_id = s.query_id \
              where {field} is not null \
              and sim.sim_q >= 0.0 \
              and pos <= 10 \
              group by sim.query_id, {field}, pos"
        cursor.execute(q)
        rows1 = cursor.fetchall()
        
        q = f"select sim.query_id,{field}, avg([pos]) as 'avg_click_pos' \
              from [ir].[dbo].[log_show] sh \
              inner join log_session s on sh.session_id = s.id \
              inner join sim_q sim on sim.log_query_id = s.query_id \
              where {field} is not null \
              and sh.is_click = 1 \
              and sim.sim_q >= 0.0 \
              group by sim.query_id, {field}"
        cursor.execute(q)
        rows2 = cursor.fetchall()
    return pd.DataFrame.from_records(rows1, columns=['query_id', field, 'pos', 'pos_clicks', 'pos_shows']), \
           pd.DataFrame.from_records(rows2, columns=['query_id', field, 'avg_click_pos'])

In [964]:
def get_stat_dict_by_q(field, field_name_df):
    click_stat_dict = {}
    click_show_stat = get_click_show_stat_by_q(field)
    click_avg_stat, click_pos_click_stat, f_click_stat, l_click_stat = get_click_stat_by_q(field)
    out10_stat = get_out10_stat_by_q(field)
    show_pos_click_stat, avg_click_pos_stat = get_pos_click_stat_by_q(field)
    
    avg_click_pos_dict = {(row['query_id'], row[field]): row['avg_click_pos'] for _, row in avg_click_pos_stat.iterrows()}
    
    f_click_dict = {(row['query_id'], row[field]): row['f_clicks'] for _, row in f_click_stat.iterrows()}
    l_click_dict = {(row['query_id'], row[field]): row['l_clicks'] for _, row in l_click_stat.iterrows()}
    
    show_pos_click_dict = {(row['query_id'], row[field], row['pos']): {c: row[c] for c in show_pos_click_stat.columns[3:]} 
                           for _, row in show_pos_click_stat.iterrows()}
    
    click_pos_click_dict = {(row['query_id'], row[field], row['pos']): 
                            {c: row[c] for c in click_pos_click_stat.columns[3:]} 
                            for _, row in click_pos_click_stat.iterrows()}
    
    click_show_dict = {(row['query_id'], row[field]): {c: row[c] for c in click_show_stat.columns[2:]} 
                       for _, row in click_show_stat.iterrows()}
    
    click_avg_dict = {(row['query_id'], row[field]): {c: row[c] for c in click_avg_stat.columns[2:]} 
                      for _, row in click_avg_stat.iterrows()}
    
    out10_dict = {(row['query_id'], row[field]): {c: row[c] for c in out10_stat.columns[2:]} 
                  for _, row in out10_stat.iterrows()}
    
    def_click_show = {c: 0 for c in click_show_stat.columns[2:]}
    def_click_avg = {c: 0 for c in click_avg_stat.columns[2:]}
    def_out10_stat = {c: 0 for c in out10_stat.columns[2:]}
    for _, row in tqdm(qdh_df.iterrows(), total=len(qdh_df)):
        key = (row.query_id, row[field_name_df])
        if not key in click_stat_dict:
            click_stat_dict[key] = {}
        else:
            continue
        click_show_by_qf = click_show_dict[key] if key in click_show_dict else def_click_show
        click_avg_by_qf = click_avg_dict[key] if key in click_avg_dict else def_click_avg
        out10_stat_by_qf = out10_dict[key] if key in out10_dict else def_out10_stat

        clicks = click_show_by_qf['clicks']
        shows = click_show_by_qf['shows']
        skips = click_show_by_qf['skips']
        not_seens = click_show_by_qf['not_seens']
        out_top10_shows = out10_stat_by_qf['out_top10_shows']
        f_clicks = f_click_dict[key] if key in f_click_dict else 0
        l_clicks = l_click_dict[key] if key in l_click_dict else 0
                     
        click_stat_dict[key]['avg_click_pos'] = avg_click_pos_dict[key] if key in avg_click_pos_dict else 0             
        click_stat_dict[key]['avg_time'] = click_avg_by_qf['avg_time']
        click_stat_dict[key]['avg_pos'] = click_show_by_qf['avg_pos']
        avg_pos = int(click_stat_dict[key]['avg_pos'])
        
        for i in range(10):
            key_pos = (row.query_id, row[field_name_df], i + 1)
            show_pos_clicks = pos_click_dict[key_pos]['pos_clicks'] \
                              if key_pos in pos_click_dict \
                              else 0
                     
            click_pos_clicks = click_pos_click_dict[key_pos]['pos_clicks'] \
                               if key_pos in click_pos_click_dict \
                               else 0
            
            pos_shows = pos_click_dict[key_pos]['pos_shows'] \
                        if key_pos in pos_click_dict \
                        else 0
            click_stat_dict[key][f"{i + 1}_show_pos_prob_click"] = show_pos_clicks / clicks if clicks > 0 else 0
            click_stat_dict[key][f"{i + 1}_click_pos_prob_click"] = click_pos_clicks / clicks if clicks > 0 else 0
            click_stat_dict[key][f"{i + 1}_ctr"] = show_pos_clicks / pos_shows if pos_shows > 0 else 0
        
        click_stat_dict[key]['first_click_prob'] = f_clicks / clicks if clicks > 0 else 0
        click_stat_dict[key]['last_click_prob'] = l_clicks / clicks if clicks > 0 else 0
        
        click_stat_dict[key]['first_ctr'] = f_clicks / shows if shows > 0 else 0
        click_stat_dict[key]['last_ctr'] = l_clicks / shows if shows > 0 else 0
                     
        click_stat_dict[key]['ctr'] = clicks / shows if shows > 0 else 0
        click_stat_dict[key]['skip_proba'] = skips / shows if shows > 0 else 0
        click_stat_dict[key]['not_seen_proba'] = not_seens / shows if shows > 0 else 0

        a = alpha[avg_pos - 1] if avg_pos <= 10 else 0
        click_stat_dict[key]['pbm'] = clicks / (shows * a) if (shows * a) > 0 else 0

        click_stat_dict[key]['out10_proba'] = out_top10_shows / shows if shows > 0 else 0

        click_stat_dict[key]['log(1+clicks)'] = np.log(1 + clicks)
        click_stat_dict[key]['log(1+shows)'] = np.log(1 + shows)
    return click_stat_dict

In [966]:
click_stat_dq = get_stat_dict_by_q('url_id', 'doc_id')

In [705]:
dh_map = {row.doc_id: row.host_id for _, row in url_data_df.iterrows()}

In [967]:
f_columns = list(click_stat_d[0].keys())
f_columns_q = list(click_stat_dq[(2, 19489)].keys())
default_dict = {c: 0 for c in f_columns}
default_q_dict = {c: 0 for c in f_columns_q}

In [968]:
from collections import defaultdict
features_dict = defaultdict(list)
for _, row in tqdm(train_df[['query_id', 'doc_id']].iterrows(), total=len(train_df)):
    host_id = dh_map[row.doc_id]
    f_h = click_stat_h[host_id] if host_id in click_stat_h else default_dict
    f_d = click_stat_d[row.doc_id] if row.doc_id in click_stat_d else default_dict
    key_hq = (row.query_id, host_id)
    f_hq = click_stat_hq[key_hq] if key_hq in click_stat_hq else default_q_dict
    key_hd = (row.query_id, row.doc_id)
    f_hd = click_stat_dq[key_hd] if key_hd in click_stat_dq else default_q_dict
    for c in f_columns:
        features_dict['h_' + c].append(f_h[c])
        features_dict['d_' + c].append(f_d[c])
    for c in f_columns_q:
        features_dict['dq_' + c].append(f_hd[c])

In [969]:
train_cs_data = {'query_id': train_df.query_id.values, 'doc_id': train_df.doc_id.values}
train_cs_data.update(features_dict)
train_cs_df = pd.DataFrame(data = train_cs_data)

In [970]:
train_cs_df.to_csv('train_cs_df.csv', index=False, sep='\t')

In [971]:
train_df = pd.read_csv('train_df.csv', sep='\t')
train_cs_df = pd.read_csv('train_cs_df.csv', sep='\t')
train_cm_smooth = pd.read_csv('train_cm_smooth.csv', sep='\t')

train_df.sort_values(by=['query_id', 'doc_id'], inplace=True)
train_cs_df.sort_values(by=['query_id', 'doc_id'], inplace=True)
train_cm_smooth.sort_values(by=['query_id', 'doc_id'], inplace=True)

In [972]:
train_cm_smooth.rename(columns={'cm_rel': "smooth_cm_rel", 
                                'sdbn_rel': "smooth_sdbn_rel", 
                                'dctr_rel': "smooth_dctr_rel"}, inplace=True)

In [973]:
train_cs_df.drop(['query_id', 'doc_id'], axis=1, inplace=True)
train_cm_smooth.drop(['query_id', 'doc_id'], axis=1, inplace=True)
train_df.drop(['query_id+doc_id'], axis=1, inplace=True)

In [974]:
train_aux_df = pd.concat([train_df, train_cs_df, train_cm_smooth], axis=1)
train_aux_df.to_csv('train_aux_2_df.csv', index=False, sep='\t')

#### Test

In [977]:
test_cs_data = {'query_id': test_df.query_id.values, 'doc_id': test_df.doc_id.values}
test_cs_df = pd.DataFrame(data = test_cs_data)

from collections import defaultdict
features_dict = defaultdict(list)
for _, row in tqdm(test_df[['query_id', 'doc_id']].iterrows(), total=len(test_df)):
    host_id = dh_map[row.doc_id]
    f_h = click_stat_h[host_id] if host_id in click_stat_h else default_dict
    f_d = click_stat_d[row.doc_id] if row.doc_id in click_stat_d else default_dict
    key_hq = (row.query_id, host_id)
    f_hq = click_stat_hq[key_hq] if key_hq in click_stat_hq else default_q_dict
    key_hd = (row.query_id, row.doc_id)
    f_hd = click_stat_dq[key_hd] if key_hd in click_stat_dq else default_q_dict
    for c in f_columns:
        features_dict['h_' + c].append(f_h[c])
        features_dict['d_' + c].append(f_d[c])
    for c in f_columns_q:
        #features_dict['hq_' + c].append(f_hq[c])
        features_dict['dq_' + c].append(f_hd[c])

test_cs_data.update(features_dict)
test_cs_df = pd.DataFrame(data = test_cs_data)

test_cs_df.to_csv('test_cs_df.csv', index=False, sep='\t')

In [978]:
test_df = pd.read_csv('test_df.csv', sep='\t')
test_cs_df = pd.read_csv('test_cs_df.csv', sep='\t')
test_cm_smooth = pd.read_csv('test_cm_smooth.csv', sep='\t')

test_df.sort_values(by=['query_id', 'doc_id'], inplace=True)
test_cs_df.sort_values(by=['query_id', 'doc_id'], inplace=True)
test_cm_smooth.sort_values(by=['query_id', 'doc_id'], inplace=True)

test_cm_smooth.rename(columns={'cm_rel': "smooth_cm_rel", 
                               'sdbn_rel': "smooth_sdbn_rel", 
                               'dctr_rel': "smooth_dctr_rel"}, inplace=True)

test_cs_df.drop(['query_id', 'doc_id'], axis=1, inplace=True)
test_cm_smooth.drop(['query_id', 'doc_id'], axis=1, inplace=True)

test_aux_df = pd.concat([test_df, test_cs_df, test_cm_smooth], axis=1)
test_aux_df.to_csv('test_aux_2_df.csv', index=False, sep='\t')

**CM**

In [257]:
click_model = CM()
start = time.time()
click_model.train(search_sessions)
end = time.time()
print("\tTrained %s click model in %i secs" % (click_model.__class__.__name__, end - start))

	Trained CM click model in 188 secs


In [261]:
rels = get_rels(train_df, click_model, predict='custom')
train_df['cm_rel'] = rels

In [262]:
rels = get_rels(test_df, click_model, predict='custom')
test_df['cm_rel'] = rels

In [296]:
from pyclick.click_models.Inference import EMInference, MLEInference

**CCM (EM)**

In [ ]:
click_model = CCM(EMInference(iter_num=1))
start = time.time()
click_model.train(search_sessions)
end = time.time()
print("\tTrained %s click model in %i secs" % (click_model.__class__.__name__, end - start))

In [ ]:
rels = get_rels(train_df, click_model, predict='orig')
train_df['ccm_rel'] = rels

In [ ]:
rels = get_rels(test_df, click_model, predict='orig')
test_df['ccm_rel'] = rels

**DCTR (MLE)**

In [275]:
click_model = DCTR()
start = time.time()
click_model.train(search_sessions)
end = time.time()
print("\tTrained %s click model in %i secs" % (click_model.__class__.__name__, end - start))

	Trained DCTR click model in 129 secs


In [276]:
rels = get_rels(train_df, click_model, predict='orig')
train_df['dctr_rel'] = rels
rels = get_rels(test_df, click_model, predict='orig')
test_df['dctr_rel'] = rels

**GCTR (MLE)**

In [277]:
click_model = GCTR()
start = time.time()
click_model.train(search_sessions)
end = time.time()
print("\tTrained %s click model in %i secs" % (click_model.__class__.__name__, end - start))

	Trained GCTR click model in 109 secs


In [278]:
rels = get_rels(train_df, click_model, predict='orig')
train_df['gctr_rel'] = rels
rels = get_rels(test_df, click_model, predict='orig')
test_df['gctr_rel'] = rels

**SDBN (MLE)**

In [279]:
click_model = SDBN()
start = time.time()
click_model.train(search_sessions)
end = time.time()
print("\tTrained %s click model in %i secs" % (click_model.__class__.__name__, end - start))

	Trained SDBN click model in 298 secs


In [280]:
rels = get_rels(train_df, click_model, predict='orig')
train_df['sdbn_rel'] = rels
rels = get_rels(test_df, click_model, predict='orig')
test_df['sdbn_rel'] = rels

**DBN (EM)**

In [304]:
click_model = DBN(EMInference(iter_num=1))
start = time.time()
click_model.train(search_sessions)
end = time.time()
print("\tTrained %s click model in %i secs" % (click_model.__class__.__name__, end - start))

	Trained DBN click model in 26187 secs


In [305]:
rels = get_rels(train_df, click_model, predict='orig')
train_df['dbn_rel'] = rels
rels = get_rels(test_df, click_model, predict='orig')
test_df['dbn_rel'] = rels

In [310]:
DBN_click_model = click_model

### Click models that depend on rank

In [285]:
search_sessions_cut = []
for s in tqdm(search_sessions, total=len(search_sessions)):
    s_cut = TaskCentricSearchSession(s.task, s.query)
    s_cut.web_results = s.web_results[:10]
    search_sessions_cut.append(s_cut)

**DCM (MLE)**

In [286]:
click_model = DCM()
start = time.time()
click_model.train(search_sessions_cut)
end = time.time()
print("\tTrained %s click model in %i secs" % (click_model.__class__.__name__, end - start))

	Trained DCM click model in 275 secs


In [287]:
rels = get_rels(train_df, click_model, predict='orig')
train_df['dcm_rel'] = rels
rels = get_rels(test_df, click_model, predict='orig')
test_df['dcm_rel'] = rels

**UBM (EM)**

In [299]:
click_model = UBM(EMInference(iter_num=1))
start = time.time()
click_model.train(search_sessions_cut)
end = time.time()
print("\tTrained %s click model in %i secs" % (click_model.__class__.__name__, end - start))

	Trained UBM click model in 677 secs


In [300]:
rels = get_rels(train_df, click_model, predict='orig')
train_df['ubm_rel'] = rels
rels = get_rels(test_df, click_model, predict='orig')
test_df['ubm_rel'] = rels

### Сохранение результатов

In [306]:
columns = ['query_id', 'doc_id', 'cm_rel', 'dctr_rel', 'gctr_rel', 'sdbn_rel', 'dcm_rel', 'ubm_rel', 'dbn_rel']

In [307]:
train_df[columns].to_csv('train_cm_df.csv', index=False, sep='\t')

In [308]:
test_df[columns].to_csv('test_cm_df.csv', index=False, sep='\t')